In [1]:
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [2]:
def cast(string):
    if string == '':
        return np.nan
    else:
        return float(string)

#Data Input
X = open('input.txt', 'r').read().splitlines()
X = [x.split(',') for x in X]
X = [[cast(a) for a in x] for x in X]

In [3]:
# hyperparameters
datum_size = len(X[0])
n_fact = 3 #no of factors
n_nutr = datum_size - n_fact
hidden_size = 100 # size of hidden layer of neurons
seq_length = len(X) # number of steps to unroll the RNN for
learning_rate = 1e-2

In [4]:
#Data Preprocessing
X_mean = np.nanmean(X,axis=0)
X_range = np.nanmax(X,axis=0) - np.nanmin(X,axis=0)
X_range[X_range == 0] = 1
X = (X - X_mean)/X_range

In [5]:
x = tf.placeholder("float", [seq_length, datum_size])
params = {
    'Wxh': tf.Variable(tf.random_normal([hidden_size, datum_size],stddev=0.01)),
    'Whh': tf.Variable(tf.random_normal([hidden_size, hidden_size],stddev = 0.01)),
    'Why': tf.Variable(tf.random_normal([n_nutr, hidden_size],stddev = 0.01)),
    'bh': tf.Variable(tf.zeros([hidden_size,1])),
    'by': tf.Variable(tf.zeros([n_nutr,1]))
}

In [10]:
def RNN(x,params):
    loss = tf.constant(0.0)
    #loss = tf.Print(loss,[loss],'init_loss:')
    h = tf.random_normal([hidden_size,1])
    x_t = tf.reshape(x[0,:],[datum_size,1])
    for t in range(seq_length-1):
        h = tf.tanh(tf.add(tf.add(tf.matmul(params['Wxh'],x_t),tf.matmul(params['Whh'],h)),params['bh']))
        y = tf.add(tf.matmul(params['Why'],h),params['by'])
        def f1(): return tf.reshape(x[t+1,:],[datum_size,1])
        def f2(): return tf.concat(0,[tf.reshape(x[t+1,:n_fact],[n_fact,1]),y])
        x_nxt = tf.cond(tf.is_nan(x[t+1,n_fact]),f2,f1)
        loss = tf.add(loss,0.5*tf.reduce_sum(tf.squared_difference(x_nxt[n_fact:,:],y)))
        #loss = tf.Print(loss,[loss],'loss:')
        x_t = x_nxt
    return loss

In [11]:
n_iter = 0
sess.run(tf.initialize_all_variables())
loss = RNN(x,params)
tf.scalar_summary('loss',loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_grads_and_vars = [(tf.clip_by_value(gv[0],-1.,1.), gv[1]) for gv in grads_and_vars]
capped_optimizer = optimizer.apply_gradients(capped_grads_and_vars)

In [12]:
merged_summary = tf.merge_all_summaries()
train_writer = tf.train.SummaryWriter('train_summary/')

In [13]:
while n_iter < 1000:
    summary,_,cost = sess.run([merged_summary,capped_optimizer,loss],feed_dict={x:X})
    train_writer.add_summary(summary,n_iter)
    if n_iter % 100 == 0:
        print('iter'+str(n_iter)+' loss:'+str(cost))
    n_iter += 1

iter0 loss:255.647
iter100 loss:41.4408
iter200 loss:18.4975


KeyboardInterrupt: 

In [13]:
sess.close()